In [215]:
import pandas as pd
import numpy as np
from statistics import mode

In [228]:
data = pd.read_csv(r"C:\Users\nalab\Desktop\Lectures\CISC351\Group\sub_info_all_1_c2.csv", header=0)
data = data.drop(columns = ["text","upvote_ratio"])
data.shape

(8985, 249)

In [229]:
vader = pd.read_csv(r"C:\Users\nalab\Desktop\Lectures\CISC351\Group\vader_1.csv", header=0)
vader.shape

(2453, 7)

In [230]:
vader["check"] = vader["positive_score"] + vader["negative_score"]
vader = vader[vader.check != 0]
vader.shape

(1877, 8)

In [231]:
vader = vader.drop(columns=['sub_author_commentkarma', 'text','positive_score','negative_score',"check"])
vader = vader.drop(vader.columns[0], axis=1)
vader.head()

,submission_id,compound_score
0,abfam2,0.9061
1,abjmvm,-0.8442
3,abn4mo,0.5719
4,abnere,0.3400
5,abp4r9,0.5106


In [234]:
merged = pd.merge(data,vader , on="submission_id")
merged.shape

(8310, 250)

In [235]:
def trust(index):    
    count = []
    a = 2 
    b = 3
    while b < 22:
        day1 = "close_{}_Y".format(a)
        day2 = "close_{}_Y".format(b)
        x = merged.loc[[index],[day1]].values[0]
        y = merged.loc[[index],[day2]].values[0]
        if y-x > 0:
            count.append(1)
        elif y-x <0:
            count.append(-1)
        else:
            count.append(0)
        a = a + 1
        b = b + 1

    z = merged.loc[[index],["compound_score"]].values[0]

    try:
        m = mode(count)
        if(z > 0 and m >0):
            return 1
        elif(z < 0 and m < 0):
           return 1
        else:
            return 0
    except: 
        return 0

In [236]:
merged["trustworthy"] = merged.index.map(trust)

In [237]:
trust_score = pd.DataFrame(merged.groupby(by=["sub_author_id"]).mean()["trustworthy"])
trust_score

,trustworthy
sub_author_id,
1022w8,0.00000
102qwv,0.00000
103h5j,0.00000
108pzv,0.50000
10abmn,0.00000
...,...
zrhkt,0.50000
zryriqd,1.00000
zsjqy,0.52439


In [238]:
trust_score.columns = ['trustscore']
trust_score

,trustscore
sub_author_id,
1022w8,0.00000
102qwv,0.00000
103h5j,0.00000
108pzv,0.50000
10abmn,0.00000
...,...
zrhkt,0.50000
zryriqd,1.00000
zsjqy,0.52439


In [242]:
merged2 = pd.merge(merged, trust_score, on="sub_author_id", how = 'left')
merged2.shape

(8310, 252)

In [247]:
for i in merged2.columns:
    print(i)

date
submission_id
num_comments
score
total_awards
sub_author_commentkarma
sub_author_id
sub_author_name
ticker
volume_20_X
adjclose_20_X
close_20_X
low_20_X
high_20_X
open_20_X
volume_19_X
adjclose_19_X
close_19_X
low_19_X
high_19_X
open_19_X
volume_18_X
adjclose_18_X
close_18_X
low_18_X
high_18_X
open_18_X
volume_17_X
adjclose_17_X
close_17_X
low_17_X
high_17_X
open_17_X
volume_16_X
adjclose_16_X
close_16_X
low_16_X
high_16_X
open_16_X
volume_15_X
adjclose_15_X
close_15_X
low_15_X
high_15_X
open_15_X
volume_14_X
adjclose_14_X
close_14_X
low_14_X
high_14_X
open_14_X
volume_13_X
adjclose_13_X
close_13_X
low_13_X
high_13_X
open_13_X
volume_12_X
adjclose_12_X
close_12_X
low_12_X
high_12_X
open_12_X
volume_11_X
adjclose_11_X
close_11_X
low_11_X
high_11_X
open_11_X
volume_10_X
adjclose_10_X
close_10_X
low_10_X
high_10_X
open_10_X
volume_9_X
adjclose_9_X
close_9_X
low_9_X
high_9_X
open_9_X
volume_8_X
adjclose_8_X
close_8_X
low_8_X
high_8_X
open_8_X
volume_7_X
adjclose_7_X
close_7_X
low_7_X


In [248]:
merged2['date'] = pd.to_datetime(merged2['date'])

In [249]:
merged3 = merged2.drop(columns = ["sub_author_id","sub_author_name","ticker"])

In [250]:
merged3.skew(axis = 0, skipna = True) 

num_comments               58.809410
score                      50.343534
total_awards               18.161515
sub_author_commentkarma     8.274041
volume_20_X                 5.720114
                             ...    
adjclose_21_Y               5.811126
volume_21_Y                 5.124413
compound_score             -1.937753
trustworthy                -0.199333
trustscore                 -0.759566
Length: 247, dtype: float64

In [251]:
merged3["sentiment_abs"] = abs(merged3["compound_score"])

In [253]:
new = merged3.corr(method="spearman")

In [254]:
new["sentiment_abs"]

num_comments              -0.248755
score                      0.331965
total_awards               0.201315
sub_author_commentkarma    0.393083
volume_20_X               -0.085198
                             ...   
volume_21_Y               -0.087824
compound_score             0.954006
trustworthy                0.038275
trustscore                 0.209837
sentiment_abs              1.000000
Name: sentiment_abs, Length: 248, dtype: float64

In [260]:
cm_results = {"num_comments" : 0.248755, "score" : 0.331965, "total_awards" : 0.201315, "sub_author_commentkarma": 0.393083, "trustscore" : 0.209837}
cf_sum = sum(cm_results.values())
cf_apply = {}
for k, v in cm_results.items():
    cf_apply[k] = v/cf_sum
print(cf_apply)

{'num_comments': 0.17961233397475007, 'score': 0.23969370845984164, 'total_awards': 0.1453585134535057, 'sub_author_commentkarma': 0.28382366214064714, 'trustscore': 0.15151178197125537}


In [257]:
import datetime
merged2['date'] = pd.to_datetime(merged2['date'])
dates = pd.Series(merged2["date"])
merged2["day"] = dates.apply(lambda x: x.timetuple().tm_yday)

In [315]:
merged2["trustscore"] = merged2["trustscore"].fillna(0)
merged2["sub_author_commentkarma"] = merged2["sub_author_commentkarma"].fillna(0)
merged2["direction"] = merged2["compound_score"].apply(lambda x: True if x > 0 else False)
merged2["abs_sentiment"] = abs(merged2["compound_score"])
merged2.head()

,date,submission_id,num_comments,score,total_awards,sub_author_commentkarma,sub_author_id,sub_author_name,ticker,volume_20_X,...,low_21_Y,close_21_Y,adjclose_21_Y,volume_21_Y,compound_score,trustworthy,trustscore,day,direction,abs_sentiment
0,2019-01-01,abfam2,9,63,0,32944.0,pnohb,jimmyjay90210,CAC,14500.0,...,39.590000,40.520000,37.886044,46200.0,0.9061,1,0.800000,1,True,0.9061
1,2019-01-01,abfam2,9,63,0,32944.0,pnohb,jimmyjay90210,AMD,79853700.0,...,22.830000,24.410000,24.410000,182575600.0,0.9061,1,0.800000,1,True,0.9061
2,2019-01-01,abfam2,9,63,0,32944.0,pnohb,jimmyjay90210,SP,56700.0,...,33.009998,33.099998,33.099998,113300.0,0.9061,1,0.800000,1,True,0.9061
3,2019-01-01,abjmvm,34,10,0,744.0,2dk1wq99,nightman_123,WSBC,65900.0,...,40.110001,40.610001,36.917271,142900.0,-0.8442,0,0.000000,1,False,0.8442
4,2019-01-01,abn4mo,23,65,0,2102.0,z68xh,Nonaluuluu,EAD,164200.0,...,7.660000,7.740000,6.377877,774000.0,0.5719,0,0.642857,1,True,0.5719


In [317]:
#3Day DataFrame

complete3 = pd.DataFrame(columns = merged2.columns)
complete3["c_comments"] = ""
complete3["c_score"] = ""
complete3["c_awards"] = ""
complete3["c_sentiment"] = ""
complete3["c_posts"] = ""
complete3

for itr in merged2.index:
    y = merged2.loc[[itr],["ticker"]].values[0][0]
    subset = merged2.loc[merged2['ticker'] == y]
    d = dates.loc[itr,]
    before_5 = d - datetime.timedelta(days=2)
    mask = (subset['date'] >= before_5) & (subset['date'] <= d)
    subset = subset.loc[mask]
    subset["day"] = subset["day"] - max(subset["day"]) + 5
    row = []
    for i in subset.columns:
        a = subset.loc[[itr],[i]].values[0][0]
        row.append(a)
    row.append(sum(subset["num_comments"]))
    row.append(sum(subset["score"]))
    row.append(sum(subset["total_awards"]))
    sday = sum(subset["day"])
    if sday == 0:
        sday = 1
    subset["day"] = subset["day"]/sday
    strust = sum(subset["trustscore"])
    if strust ==0:
        strust = 1
    subset["trustscore"] = subset["trustscore"]/strust
    skarma = sum(subset["sub_author_commentkarma"])
    if skarma ==0:
        skarma = 1
    subset["sub_author_commentkarma"] = subset["sub_author_commentkarma"]/skarma
    saward = sum(subset["total_awards"])
    if saward ==0:
        saward = 1
    subset["total_awards"] = subset["total_awards"]/saward
    sscore = sum(subset["score"])
    if sscore ==0:
        sscore = 1
    subset["score"] = subset["score"]/sscore
    scomment = sum(subset["num_comments"])
    if scomment ==0:
        scomment = 1
    subset["num_comments"] = subset["num_comments"]/scomment
    subset["adjScore"] = subset["abs_sentiment"] + subset["day"] + subset["trustscore"]*cf_apply.get("trustscore") + subset["sub_author_commentkarma"]*cf_apply.get("sub_author_commentkarma") + subset["total_awards"]*cf_apply.get("total_awards") + subset["score"]*cf_apply.get("score") + subset["num_comments"]*cf_apply.get("num_comments")
    row.append(subset["adjScore"].mean())
    row.append(len(subset))
    row_series = pd.Series(row, index = complete3.columns)
    complete3 = complete3.append(row_series, ignore_index=True)
    print("Completed Iteration", itr)

Completed Iteration 0
Completed Iteration 1
Completed Iteration 2
Completed Iteration 3
Completed Iteration 4
Completed Iteration 5
Completed Iteration 6
Completed Iteration 7
Completed Iteration 8
Completed Iteration 9
Completed Iteration 10
Completed Iteration 11
Completed Iteration 12
Completed Iteration 13
Completed Iteration 14
Completed Iteration 15
Completed Iteration 16
Completed Iteration 17
Completed Iteration 18
Completed Iteration 19
Completed Iteration 20
Completed Iteration 21
Completed Iteration 22
Completed Iteration 23
Completed Iteration 24
Completed Iteration 25
Completed Iteration 26
Completed Iteration 27
Completed Iteration 28
Completed Iteration 29
Completed Iteration 30
Completed Iteration 31
Completed Iteration 32
Completed Iteration 33
Completed Iteration 34
Completed Iteration 35
Completed Iteration 36
Completed Iteration 37
Completed Iteration 38
Completed Iteration 39
Completed Iteration 40
Completed Iteration 41
Completed Iteration 42
Completed Iteration 4

KeyboardInterrupt: 

In [320]:
#To make the 5 day and 1 day datasets, just change the number of days to 4, and 0 respectively. 